# **LAB FOUR: RELAXATION AND CONTRAST**


>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
LAB_USER_NAME = 'Peder'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [2]:
import panel as pn
import sys
import os
import yaml
import numpy as np

# add libraries to path so they can be imported later
sys.path.append('../../../')

from matipo import Sequence, SEQUENCE_DIR, GLOBALS_DIR, DATA_DIR
from bokeh.palettes import Viridis
from ROI_stats_image_plot import ROIStatsImagePlot
from multi_dot_plot import MultiDotPlot
from matipo.util.plots import PLOT_COLORS
import asyncio

from matipo.util.decimation import decimate
from matipo.util.fft import fft_reconstruction
from matipo.util.etl import deinterlace

from experiment import BaseExperiment # load before pn.extension() for stylesheet changes on panel < 1.0
from experiment.plot import SignalPlot, SpectrumPlot

pn.extension()

LAB_DIR = os.path.join(DATA_DIR, LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

Data will be saved to /home/data/Peder


## 1. Background

For MR imaging to be useful in diagnostics we need to be able to differentiate between different types of soft tissue and fluid in the image, e.g. identifying a tumour in the brain. These different soft tissues may have similar amounts of water, and therefore proton density, which makes them difficult to distinguish.

So far, the contrast you have been seeing in images has been only based on proton density. Areas of the phantoms with more hydrogen nuclei (like fluid or silicone) appear brighter while areas with less hydrogen nuclei (air or solids) appear darker. But how do you differentiate between two materials that have the same proton density?

This requires a closer look at the topic of relaxation. Tissue density, tissue structure, and chemical composition, all affect a material’s relaxation times. The different relaxation times of materials and tissues is routinely used to create contrast in MR imaging, and in some cases the relaxation times are delibrately altered with chemical contrast agents.

### 1.1. Recap on Net Magnetization

<figure style="float: right;">
<img src="images/M_vector.png" width="350">
<center><figcaption style="width: 350px;">Figure 1: Net Magnetization Vector</figcaption></center>
</figure>

Since NMR is a quantum phenomenon, it is important to recap how individual spins contribute to the net magnetization vector.  (But never fear, quantum mechanics is actually not required to understand MRI.)

The contribution of all the quantum spins' magnetic moments ( $\vec\mu$) results in a vector that describes the net direction and magnitude of these spins - this is called the Net Magnetization Vector ($\vec M$). When placed in the magnetic field ($\vec B_0$), not all the spins will align with the field, but at thermal equilibrium their net value ($\vec M$) will align. $\vec M$ will precess around $\vec B_0$ at the Larmor frequency (as described in Lab One). When we add energy to the system, using a magnetic field oscillating at the Larmor frequency ($\vec B_1$), the tip angle of ($\vec M$) can be changed.

**Relaxation** is the process of ($\vec M$) returning back to its thermal equilibrium state in alignment with $\vec B_0$.


The different types of relaxation fundamentally occur in different dimensions, so it is helpful to split $\vec M$ into two components (Fig.1): the longitudinal component ($M_z$) and the transverse component ($M_{xy} = M_x + i M_y$). Each of these components undergo relaxation at different rates and through different mechanisms.

### 1.2. Longitudinal (T1) Relaxation

<figure style="float: right;">
<img src="images/T1.png" width="350">
<center><figcaption style="width: 350px;">Figure 2: $T_1$ Relaxation Curve</figcaption></center>
</figure>

Longitudinal relaxation is caused by a transfer of energy from the spin system to the external environment. Directly after a 90 degree pulse (which tips $\vec M$ in the XY plane), energy has been transferred into the spin system and the longitudinal component ($ M_z$) is close to zero. As the spin system loses this energy and returns to thermal equilibrium, the longitudinal component ($ M_z$) regrows. Figure 2 shows this relaxation after a 90 pulse. 

This energy transfer is not spontaneous, however. To add energy to the spin system, $\vec B_1$ must oscillate at the Larmor frequency. Similarly, energy emissions must be stimulated by a transverse magnetic field fluctuating near the Larmour frequency. These field fluctuations occur due to molecular motion of nearby nuclei or electrons. The rate at which individual spins encounter fields fluctuating at the frequency needed for energy transfer is highly dependent on the molecular structure of the material or tissue being measured. This results in a wide range of longitudinal relaxation times across different materials. The time constant that describes the longitudinal relaxation of a material  is $T_1$. 

$$M_z(t) = M_0 \cdot (1-e^{-t/T_1}) + M_z(0) \cdot e^{-t/T_1} \tag{1}$$

where $M_0$ is the equilibrium magnetization (after a sufficiently long time, $M_z(t \rightarrow \infty) = M_0$) and $M_z(0)$ is the initial longitudinal magnetization.


### 1.3. Transverse (T2) Relaxation

<figure style="float: right;">
<img src="images/T2.png" width="350">
<center><figcaption style="width: 350px;">Figure 3: $T_2$ Relaxation Curve</figcaption></center>
</figure>

Transverse relaxation is caused by the dephasing of the individual spins. Directly after a 90 pulse, the spins are almost entirely in phase and $ M_{xy}$ is at its maximum. As phase coherence is lost, the transverse component ($ M_{xy}$) returns to zero (Fig.3).

Transverse relaxation occurs more rapidly than longitudinal relaxation. This is because the energy emissions that cause longitudinal relaxation randomly change the direction and phase of a magnetic moment resulting in dephasing in the transverse plane. However, dephasing can also occur without energy transfer into the environment. For example, dipole-dipole interactions cause dephasing without the energy release that affects $ M_z$. The time constant that describes the transverse relaxation properties of a material is called $T_2$. 

$T_2$ is not to be confused with $T_2^*$. $T_2^*$ decay is faster than $T_2$ decay because it consistents of $T_2$ decay effects and additional decay due to $\vec B_0$ field inhomogeneities. However, the dephasing that causes the additional decay in $T_2^*$ can be easily reversed and the magnitude of the transverse component restored (see Lab 1 for a more in depth explanation of $T_2^*$). In contrast, the dephasing and loss of signal amplitude associated with $T_2$ relaxation can not be reversed.

$$M_{xy}(t) = M_{xy}(0) \cdot e^{-t/T_2} \tag{2}$$

$M_{xy}(0)$ is the initial magnetization in the XY plane after the RF pulse.

### 1.4. Signal Amplitude
So, now we know how $T_1$ and $T_2$ relaxation work. How do we use them to control the contrast in an image? This is done by carefully selecting pulse sequence parameters to maximise the difference between the signal amplitudes of different materials. High signal amplitude will result in a material showing up brighter on an image and low signal amplitude appear darker. 

Signal amplitude is proportional to the magnitude of $\vec M_{xy}$ during acquisition and is controlled by three key factors (illustrated in Fig.4):

1) The equilibrium magnitude of $\vec M$, which is $M_0$. This is determined by the proton density. Materials with higher proton density will have more quantum spins to contribute to $\vec M_z$. 

2) The amount of $ M_z$ that is allowed to recover before it is tipped into the transverse plane to become the ($ M_{xy}$) component. This is determined by both the time constant of longitudinal recovery ($T_1$) and the amount of recovery time ($T_R$) allowed before the next RF pulse as given in Equation 1.

3) The amount the $ M_{xy}$ component is allowed to decay before acquisition occurs. This is determined by the time constant of transverse decay ($T_2$) and the amount of time the signal is allowed to decay before acquisition ($T_E$) as given in Equation 2.


<center><img src="images/signal_amp.png" width="1200"></center>
<center><figcaption style="width: 600px;">Figure 4: Key Factors that Contribute to Signal Amplitude.  Note that in (1) it should be $M_0$, not $M_z(0)$ </figcaption></center>



While proton density and the rate of $T_1$ and $T_2$ decay are intrinsic properties of each material, we do have control over the amount of time $ M_z$ is given to recover before the next RF pulse and the amount of time $ M_{xy}$ is allowed to decay before acquisition. Figure 5 shows the growth and decay of the $\vec M_z$ and $\vec M_{xy}$ components during a spin echo experiment. In this pulse sequence, the length of $T_R$ (the time between 90 pulses) controls how much $ M_z$ is allowed to recover before it is flipped into the transverse plane and the length of $T_E$ (the time between the 90 pulse and the echo) controls how much of $ M_{xy}$ is allowed to decay before acquisition occurs. 

<center><img src="images/SpinEcho_T1_T2.png" width="900"></center>
<center><figcaption style="width: 600px;">Figure 5: Spin Echo Pulse Sequence</figcaption></center>



The following sections will examine these two parameters in more depth and explore how they can be used to control different types of image contrast. 


## 2. TR
Figure 6 shows the behaviour of two samples that have the same proton density and $T_2$ properties but have different $T_1$ properties. 
 
1) **When $T_R$ is long**, the $ M_z$ components of both the samples are able to recover fully before being flipped into the transverse plane. Since they have the same $T_2$ decay properties, they will have the same signal amplitude at acquisition. In the image, **samples with different $T_1$ recovery times will have similar brightness and can not be easily distinguished**. 


2) **When $T_R$ is short**, the sample with a longer $T_1$ recovery time does not have time to fully recover before being tipped into the transverse plane. Although both samples have the same $T_2$ decay properties, the one shown in yellow has a smaller initial $ M_{xy}$ component and, therefore, will have lower signal amplitude at acquisition. In the image, **samples with a longer $T_1$ recovery times will appear darker than those with shorter $T_1$**. 

Therefore a short $T_R$ can be used to highlight areas with a short $T_1$ recovery time.

<center><img src="images/SpinEcho_TR.png" width="1200"></center>
<center><figcaption style="width: 600px;">Figure 6: Effect of $T_R$ on Signal Amplitude</figcaption></center>


### Note: SNR and CNR

Signal to noise ratio (SNR) will be defined as<sup>1</sup>:

$$ \mathrm{SNR} = \frac{\mu_S}{\sigma_N} \tag{3}$$

where $\mu_S$ is the mean signal and $\sigma_N$ is the standard deviation of the noise.
Contrast to noise ratio will be defined as:

$$ \mathrm{CNR} = \frac{|\mu_A - \mu_B|}{\sigma_N} \tag{4}$$

where $\mu_A$ and $\mu_B$ are the mean signal values of two regions of interest.

<sup>1</sup> This is the definition generally used to describe images, but is different from the power-based SNR definition normally used in engineering.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1:**
> The goal of this task is to find the $T_R$ value that maximises the contrast between two different materials. In quantitative terms this means maximising the CNR as given in Equation 4.
> 
> 1. Insert the pie phantom at the correct depth.
> 2. Run the experiment below and observe the alignment of ROI circles with the wedges in the image.
> 3. Rotate the phantom and rerun the experiment until the two wedges are aligned with two of the ROI circles. The empty ROI circle will be used to measure the background noise.
> 4. Clear the Amplitude vs $T_R$ History plot with the "Clear History" button.
> 5. Run the experiment with a variety of *Rep. Time* ($T_R$) values.
> 6. What happens if $T_R$ is short? How about when it is long?
> 7. At which $T_R$ value are the two wedges most visually distinct?
> 8. Which $T_R$ value gives the highest CNR, and what is this CNR value? (Hint: The noise $\sigma_N$ should be constant, so CNR is maximised when the difference $|\mu_A - \mu_B|$ is maximised, which can be seen in the *Amplitude vs $T_R$* plot)
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:


class VariableTRImageExperiment(BaseExperiment):
    title='Variable TR Imaging Experiment'
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Image"),color_palette=PLOT_COLORS),
        'roi_history': MultiDotPlot(figure_opts=dict(
            title="Amplitude vs TR History",
            x_axis_label='Rep. Time (s)',
            y_axis_label='Amplitude')
            ,color_palette=PLOT_COLORS)
    }

    data_history = {
        'TR': [],
        'Amp_1': [],
        'Amp_2': [],
        'Amp_3': []
    }
    
    roi=dict(
        center_x=[-1.73,1.73,0],
        center_y=[1,1,-2],
        radius=[0.75,0.75,0.75]
    )
    
    DECIMATION = 4
    FOV = 10
    
    t_rep_input = pn.widgets.DiscreteSlider(name='Rep. Time (s)', options=[0.025, 0.05, 0.1, 0.2, 0.5, 1.0], value=0.2, width=200)
    
    def clear_history(self):
        self.data_history = {
            'TR': [],
            'Amp_1': [],
            'Amp_2': [],
            'Amp_3': [],
        }
        asyncio.ensure_future(self._update_plots_wrapper()) # TODO: add utility method to BaseExperiment that does this

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'shims.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
        self.seq.setpar(
            t_rep = self.t_rep_input.value,
            n_scans=2,

            n_ETL=8,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 64 phase steps
            n_phase_2=64,
            g_phase_2=(0, 1.0, 0),

            t_dw=4e-6,
            n_samples=64*self.DECIMATION,
            g_read=(-0.7, 0, 0),
            
            enable_dummy_run=True
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        # set minimum t_echo
        self.seq.setpar(t_echo=self.seq.par.t_180 + 4*self.seq.par.t_grad_ramp + 2*self.seq.par.t_phase + self.seq.par.t_read + 2*1e-7)
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])
        if self.progress.value == self.progress.max:
            self.data_history['TR'].append(self.seq.par.t_rep)
            self.data_history['Amp_1'].append(self.plots['image'].roi_mean[0])
            self.data_history['Amp_2'].append(self.plots['image'].roi_mean[1])
            self.data_history['Amp_3'].append(self.plots['image'].roi_mean[2])
            self.plots['roi_history'].update(dict(
                Amp_1 = dict(x=self.data_history['TR'],y= self.data_history['Amp_1']),
                Amp_2 = dict(x=self.data_history['TR'],y= self.data_history['Amp_2']),
                Amp_3 = dict(x=self.data_history['TR'],y= self.data_history['Amp_3'])
            ))
            

exp1 = VariableTRImageExperiment()
exp1.btn_clear_history = pn.widgets.Button(name='Clear History', button_type='danger', width=200, margin=(22,10))
exp1.btn_clear_history.on_click(lambda e: exp1.clear_history())
pn.Column(
    pn.Row(exp1.t_rep_input, pn.layout.HSpacer(), exp1.btn_clear_history, sizing_mode='stretch_width'),
    exp1(),
sizing_mode="stretch_both").servable()

Column(sizing_mode='stretch_both')
    [0] Row(sizing_mode='stretch_width')
        [0] DiscreteSlider(name='Rep. Time (s)', options=[0.025, 0.05, ...], value=0.2, width=200)
        [1] HSpacer()
        [2] Button(button_type='danger', margin=(22, 10), name='Clear History', width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] HTML(str, sizing_mode='stretch_width')
        [1] Row(min_height=400, sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [2] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=32, margin=0, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 3. TE
Figure 7 shows the behaviour of two samples that have the same proton density and $T_1$ properties but different $T_2$ properties.

1) **When $T_E$ is short**, the $ M_{xy}$ components of both samples do not have much time to decay and, as a result, are close to their maximum values when acquisition occurs. In the image, **samples with different $T_2$ recovery times will have similar brightness and can not be easily distinguished**. 

2) **When $T_E$ is long**, the sample with the shorter $T_2$ decay time will decay significantly before acquisition occurs. In the image, **samples with shorter $T_2$ relaxation times will appear darker than those with longer $T_2$ relaxation times**.

Therefore a long $T_E$ can be used to highlight areas with a long $T_2$ decay time.

<center><img src="images/SpinEcho_TE.png" width="900"></center>
<center><figcaption style="width: 600px;">Figure 7: Effect of $T_E$ on Signal Amplitude</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1:**
> The goal of this task is to find the $T_E$ value that maximises the contrast between two different materials. In quantitative terms this means maximising the CNR as given in Equation 4.
> 
> 1. Insert the pie phantom at the correct depth.
> 2. Run the experiment below and observe the alignment of ROI circles with the wedges in the image.
> 3. Rotate the phantom and rerun the experiment until the two wedges are aligned with two of the ROI circles. The empty ROI circle will be used to measure the background noise.
> 4. Clear the Amplitude vs $T_E$ History plot with the "Clear History" button.
> 5. Run the experiment with a variety of *Echo Time* ($T_E$) values.
> 6. What happens if $T_E$ is short? How about when it is long?
> 7. At which $T_E$ value are the two wedges most visually distinct?
> 8. Which $T_E$ value gives the highest CNR, and what is this CNR value? (Hint: The noise $\sigma_N$ should be constant, so CNR is maximised when the difference $|\mu_A - \mu_B|$ is maximised, which can be seen in the *Amplitude vs $T_E$* plot)
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# NOTE: There are strange CPMG effects at certain echo times (4-5ms), so this experiment has been set up with a fixed echo time (3 ms)
# with ETL varying the effective TE

class VariableTEImageExperiment(BaseExperiment):
    title='Variable TE Imaging Experiment'
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Image"),color_palette=PLOT_COLORS),
        'roi_history': MultiDotPlot(figure_opts=dict(
            title="Amplitude vs TE History",
            x_axis_label='Echo Time (s)',
            y_axis_label='Amplitude')
            ,color_palette=PLOT_COLORS)
    }

    data_history = {
        'TE': [],
        'Amp_1': [],
        'Amp_2': [],
        'Amp_3': [],
    }
    
    roi=dict(
        center_x=[-1.73,1.73,0],
        center_y=[1,1,-2],
        radius=[0.75,0.75,0.75]
    )
    
    DECIMATION = 4
    FOV = 10
    
    T_ECHO_MS = 3
    TE_input = pn.widgets.DiscreteSlider(name='Echo Time (ms)', options=(T_ECHO_MS*np.array([2,4,8,16,32,64,128])).tolist(), value=T_ECHO_MS*32)
    n_ETL_display = pn.widgets.FloatInput(name='Echo Train Length', value=60, width=200, disabled=True)
    
    def clear_history(self):
        self.data_history = {
            'TE': [],
            'Amp_1': [],
            'Amp_2': [],
            'Amp_3': [],
        }
        asyncio.ensure_future(self._update_plots_wrapper()) # TODO: add utility method to BaseExperiment that does this

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'shims.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
            
        self.seq.setpar(
            t_rep = 1,
            n_scans=2,

            n_ETL=64,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 60 phase steps
            n_phase_2=64,
            g_phase_2=(0, 0.8, 0),

            t_dw=4e-6,
            n_samples=64*self.DECIMATION,
            g_read=(-0.7, 0, 0),
            
            t_echo=self.T_ECHO_MS*1e-3,
            
            enable_dummy_run=False
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        
        self.t_echo_eff = self.TE_input.value*1e-3
        # multiply t_echo by 2 until n_ETL works
        n_ETL = int(round(2*(self.t_echo_eff/self.seq.par.t_echo)))
        while n_ETL>self.seq.par.n_phase_2:
            self.seq.setpar(t_echo=self.seq.par.t_echo*2)
            n_ETL = int(round(2*(self.t_echo_eff/self.seq.par.t_echo)))
        self.n_ETL_display.value = n_ETL
        self.seq.setpar(n_ETL=n_ETL)
         
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])
        if self.progress.value == self.progress.max:
            self.data_history['TE'].append(self.t_echo_eff)
            self.data_history['Amp_1'].append(self.plots['image'].roi_mean[0])
            self.data_history['Amp_2'].append(self.plots['image'].roi_mean[1])
            self.data_history['Amp_3'].append(self.plots['image'].roi_mean[2])
            self.plots['roi_history'].update(dict(
                Amp_1 = dict(x=self.data_history['TE'],y= self.data_history['Amp_1']),
                Amp_2 = dict(x=self.data_history['TE'],y= self.data_history['Amp_2']),
                Amp_3 = dict(x=self.data_history['TE'],y= self.data_history['Amp_3']),
            ))

exp2 = VariableTEImageExperiment()
exp2.btn_clear_history = pn.widgets.Button(name='Clear History', button_type='danger', width=200, margin=(22,10))
exp2.btn_clear_history.on_click(lambda e: exp2.clear_history())

pn.Column(
    pn.Row(exp2.TE_input, pn.layout.HSpacer(), exp2.btn_clear_history, sizing_mode='stretch_width'),
    exp2(),
    sizing_mode="stretch_both"
).servable()

Column(sizing_mode='stretch_both')
    [0] Row(sizing_mode='stretch_width')
        [0] DiscreteSlider(formatter='%d', name='Echo Time (ms)', options=[6, 12, 24, 48, ...], value=96)
        [1] HSpacer()
        [2] Button(button_type='danger', margin=(22, 10), name='Clear History', width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] HTML(str, sizing_mode='stretch_width')
        [1] Row(min_height=400, sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [2] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=32, margin=0, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 4. Contrast Imaging

It is important to note that $T_1$ and $T_2$ weighted imaging produce opposite contrast: With $T_1$ weighted imaging areas with short relaxation are highlighted, whereas with $T_2$ weighted imaging areas with long relaxation are highlighted. The $T_1$ and $T_2$ properties of a sample tend to be related, so the choice of $T_1$ or $T_2$ weighting may be decided by which part of object under study should be highlighted. Also $T_1$ and $T_2$ weighting can technically be used together, but generally are not, as suppressing both short and long relaxation times results in poor SNR and contrast.

| $T_R$ | $T_E$ | Weighting | Result |
| --- | ----------- | ----------- | ----------- | 
| Long | Short | Proton Density Weighted | high SNR, low contrast |
| Short | Short |  T1 Weighted | Short T1 highlighted |
| Long | Long | T2 Weighted | Long T2 highlighted |
| Short | Long |  T1 & T2 Weighted | low SNR |

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.1:**
> The goal of this task is to change the contrast of the image to reveal the hidden shapes.
> 
> 1. Insert the shapes phantom at the correct depth.
> 2. Run the experiment and observe the image.
> 3. Adjust the parameters (if necessary) to achieve a proton density weighted image where all the parts of the phantom look equally bright. Record the $T_R$ and $T_E$ values and save the image plot.
> 4. Based on your previous results, adjust $T_R$ and $T_E$ to achieve maximum $T_1$ contrast and minimum $T_2$ contrast. Record the $T_R$ and $T_E$ values and save the image plot.
> 5. Now adjust $T_R$ and $T_E$ to achieve maximum $T_2$ contrast and minimum $T_1$ contrast. Record the $T_R$ and $T_E$ values and save the image plot.
>
> Note: Some combinations of $T_R$ and $T_E$ are not possible, $T_R$ must be < 2\*$T_E$ for this experiment.
>
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
class ContrastImageExperiment(BaseExperiment):
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Image"),color_palette=PLOT_COLORS),
    }
    
    roi=dict(
        center_x=[],
        center_y=[],
        radius=[]
    )
    
    DECIMATION = 4
    FOV = 12
    T_ECHO_MS = 3
    
    t_rep_input = pn.widgets.DiscreteSlider(name='Rep. Time (s)', options=[0.03, 0.05, 0.1, 0.2, 0.5, 1.0], value=0.5, width=200)
    TE_input = pn.widgets.DiscreteSlider(name='Echo Time (ms)', options=(T_ECHO_MS*np.array([2,4,8,16,32,64,128])).tolist(), value=T_ECHO_MS*4)
    n_ETL_display = pn.widgets.FloatInput(name='Echo Train Length', value=60, width=200, disabled=True)

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'shims.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
        self.seq.setpar(
            t_rep = self.t_rep_input.value,
            n_scans=2,

            n_ETL=64,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 64 phase steps
            n_phase_2=64,
            g_phase_2=(0, 1.0, 0),

            t_dw=4e-6,
            n_samples=64*self.DECIMATION,
            g_read=(-0.7, 0, 0),
            
            t_echo=self.T_ECHO_MS*1e-3,
            
            enable_dummy_run=True
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        
        self.t_echo_eff = self.TE_input.value*1e-3
        # multiply t_echo by 2 until n_ETL works
        n_ETL = int(round(2*(self.t_echo_eff/self.seq.par.t_echo)))
        while n_ETL>self.seq.par.n_phase_2:
            self.seq.setpar(t_echo=self.seq.par.t_echo*2)
            n_ETL = int(round(2*(self.t_echo_eff/self.seq.par.t_echo)))
        
        self.n_ETL_display.value = n_ETL
        self.seq.setpar(n_ETL=n_ETL)
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])

exp3 = ContrastImageExperiment()
pn.Column(
    pn.Row(exp3.t_rep_input, exp3.TE_input),
    exp3(),
    sizing_mode="stretch_both"
).servable()

Column(sizing_mode='stretch_both')
    [0] Row
        [0] DiscreteSlider(name='Rep. Time (s)', options=[0.03, 0.05, 0.1, ...], value=0.5, width=200)
        [1] DiscreteSlider(formatter='%d', name='Echo Time (ms)', options=[6, 12, 24, 48, ...], value=12)
    [1] Column(sizing_mode='stretch_both')
        [0] HTML(str, sizing_mode='stretch_width')
        [1] Row(min_height=400, sizing_mode='stretch_both')
            [0] Bokeh(Figure)
        [2] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=32, margin=0, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')